In [1]:
import time
import torch
import numpy as np

from scipy.io import wavfile
import scipy.signal as sps

from transformers import AutoProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

from TTS.api import TTS

/home/qywu/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-02 16:10:25.520033: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 16:10:25.982717: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-02 16:10:25.982759: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dl

In [42]:
class ASR:
    def __init__(self) -> None:
        model_name = "openai/whisper-medium"
        self.processor = AutoProcessor.from_pretrained(model_name)
        self.model = WhisperForConditionalGeneration.from_pretrained(model_name)
        self.model_sample_rate = 16000
        self.device = torch.device("cpu")

    def __call__(self, data, sample_rate=16000) -> str:
        """
        Args:
            data: PCM float32 format
            sample_rate: the sample rate of data
        """
        # first, resample the data to the model's sample_rate
        number_of_samples = round(len(data) * float(self.model_sample_rate) / sample_rate)
        data = sps.resample(data, number_of_samples)

        # genearte text
        inputs = self.processor(data, return_tensors="pt", sampling_rate=self.model_sample_rate)
        input_features = inputs.input_features.to(self.device)
        generated_ids = self.model.generate(inputs=input_features)
        text = self.processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

        return text 

    def to(self, device):
        self.model = self.model.to(device)
        self.device = device
        return self


In [43]:
model = ASR()

In [50]:
# specify the running device
device = torch.device("cuda")
model = model.to(device)

In [51]:
rate, data = wavfile.read("test.wav")
# requires float32 data
data = data.astype(np.float32) / 32768.0

In [52]:
text = model(data, sample_rate=rate)
print(text)

The birch canoe slid on the smooth planks. Glue the sheet to the dark blue background. It is easy to tell the depth of a well. These days a chicken leg is a rare dish. Rice is often served in round bowls. The juice of lemons makes fine punch. The box was thrown beside the park truck. The hogs were fed chopped corn and garbage. Four hours of steady work faced us.
